<a href="https://colab.research.google.com/github/rohitshekar/mediaproject/blob/main/GenderAge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%cd /content/

/content


In [ ]:
%mkdir images
%cd images

/content/images


In [ ]:
%cd images

[Errno 2] No such file or directory: 'images'
/content/images


In [ ]:
!kaggle datasets download -d jangedoo/utkface-new

100% 330M/331M [00:17<00:00, 23.9MB/s]
100% 331M/331M [00:17<00:00, 19.7MB/s]


In [ ]:
!unzip /content/images/utkface-new.zip

In [ ]:
import os
a=os.listdir('/content/images/utkface_aligned_cropped')[:5]
print(a)

['crop_part1', 'UTKFace']


In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os

In [ ]:
base_dir='/content/images/UTKFace'

In [ ]:
base_dir='/content/images/utkface_aligned_cropped/UTKFace'

In [ ]:
files=os.listdir(base_dir)

In [ ]:
len(files)

23708

In [ ]:
files[0].split('_')

['68', '0', '0', '20170119212026025.jpg.chip.jpg']

In [ ]:
%cd /content

/content


In [ ]:
image_paths=[]
gender=[]
age=[]
ethincity=[]
for i in tqdm(files):
  img_path=os.path.join(base_dir,i)
  temp=i.split('_')
  if len(temp)==4:
   image_paths.append(img_path)
   age.append(temp[0])
   gender.append(temp[1])
   ethincity.append(temp[2])


  0%|          | 0/23708 [00:00<?, ?it/s]

In [ ]:
for i in tqdm(files):
  img_path=os.path.join(base_dir,i)
  temp=i.split('_')
  if len(temp)==4:
   image_paths.append(img_path)
   age.append(temp[0])
   gender.append(temp[1])
   ethincity.append(temp[2])

  0%|          | 0/23708 [00:00<?, ?it/s]

In [ ]:
print(set(ethincity))

{'4', '0', '1', '2', '3'}


In [ ]:
import tensorflow as tf

In [ ]:
a=pd.DataFrame()

In [ ]:
a['img_path'],a['gender'],a['age'],a['ethincity']=image_paths,gender,age,ethincity

In [ ]:
set(age)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import cv2

In [ ]:
def extract(images):
  features=[]
  for i in tqdm(images):
    img=cv2.imread(i)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(64,64))
    harrcas=cv2.CascadeClassifier('facedetect.xml')
    rect=harrcas.detectMultiScale(img,scaleFactor=1.2,minNeighbors=2)
    for x,y,w,h in rect:
       img=img[y:y+h,x:x+w]
       img=img[y:y+h,x:x+w]
       img = np.array(img)
       img=img/225.0
       features.append(img)
  features = np.array(features)
  return features

In [ ]:
from PIL import Image
features=extract(a['img_path'])

  0%|          | 0/47410 [00:00<?, ?it/s]

<ipython-input-79-88e3c4ccfd0a>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(features)


In [ ]:
features=np.array(features)
features=features.reshape(len(features),64,64,3)

In [ ]:
y_gender=np.array(a['gender'])
y_age=np.array(a['age'])
y_ethincity=np.array(a['ethincity'])


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input

In [ ]:
inputs = Input((64,64,3))
# convolutional layers
conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu') (inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2)) (conv_1)
conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu') (maxp_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2)) (conv_2)
conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu') (maxp_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2)) (conv_3)
conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu') (maxp_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2)) (conv_4)


flatten = Flatten() (maxp_4)
dropout_1 = Dropout(0.2) (flatten)
dropout_2 = Dropout(0.2) (flatten)
dropout_3 = Dropout(0.2) (flatten)


# fully connected layers
dense_1 = Dense(256, activation='relu') (dropout_1)
dense_2 = Dense(256, activation='relu') (dropout_2)
dense_3 = Dense(256, activation='relu') (dropout_3)

dropout_4 = Dropout(0.2) (dense_1)
dropout_5 = Dropout(0.2) (dense_2)
dropout_6 = Dropout(0.2) (dense_3)


output_1 = Dense(1, activation='sigmoid', name='gender_out') (dropout_4)
output_2 = Dense(4, activation='softmax', name='age_out') (dropout_5)
output_3 = Dense(5, activation='softmax', name='et_out') (dropout_6)


model = Model(inputs=[inputs], outputs=[output_1,output_2,output_3])

model.compile(loss= ['binary_crossentropy','categorical_crossentropy','categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x=features, y=[y_gender,y_age,y_ethincity], batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
model.save('/content/drive/MyDrive/ageet3.h5',save_format="h5")

In [ ]:
model=tf.keras.saving.load_model('/content/ageet3.h5')

In [ ]:
import cv2
img=cv2.imread('/content/IMG_20210623_214150_4.jpg')
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
harrcas=cv2.CascadeClassifier('/content/facedetect.xml')
rect=harrcas.detectMultiScale(img,scaleFactor=1.2,minNeighbors=2)
for x,y,w,h in rect:
   img=img[y:y+h,x:x+w]
img =cv2.resize(img,(64,64))
img=np.array(img)
img=img/225.0
img=np.expand_dims(img,axis=0)
res=model.predict(img)
result=res[0][0]
print(result,np.argmax(res[1][0]),np.argmax(res[2][0]))


1/1 [==============================] - 0s 59ms/step
[0.03201] 0 4


In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
a=cv2.imread('/content/IMG_20210623_214150_4.jpg')
plt.imshow(a)
plt.show()

In [ ]:
y_age=[int(x) for x in y_age]




In [ ]:
y_gender=[int(x) for x in y_gender]

In [ ]:
y_gender=np.array(y_gender)
y_age=np.array(y_age)
y_ethincity=np.array(y_ethincity)

In [ ]:
for i in range(len(y_age)):
  if y_age[i]>=0 and y_age[i]<25:
    y_age[i]=0
  elif y_age[i]>=25 and y_age[i]<40:
    y_age[i]=1
  elif y_age[i]>=40and y_age[i]<=60:
    y_age[i]=2
  else:
    y_age[i]=3


In [ ]:
y_age=[str(x) for x in y_age]


In [ ]:
y_age[0]

array([0., 0., 0., 1.], dtype=float32)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_age=to_categorical(y_age)


In [ ]:
y_ethincity=[int(x) for x in y_ethincity]


In [ ]:
y_ethincity=to_categorical(y_ethincity)

In [ ]:
y_gender

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils
encoder_age = LabelEncoder()
encoder_age.fit(y_age)
encoded_Y = encoder_age.transform(y_age)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_yage = utils.to_categorical(encoded_Y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils
encoder_eth = LabelEncoder()
encoder_eth.fit(y_ethincity)
encoded_Y = encoder_eth.transform(y_ethincity)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_yet = utils.to_categorical(encoded_Y)

In [ ]:
dummy_yet[0]

array([0., 0., 0., 1.], dtype=float32)

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('/content/images/UTKFace/53_0_0_20170120223133507.jpg.chip.jpg', target_size = (64,64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result_age(result),result_et(result))

1/1 [==============================] - 0s 20ms/step
0
50+ White


In [ ]:
def result_age(result):
 dic={
    0:'0-18',
    1:'18-30',
    2:'30-50',
    3:'50+'
 }
 c=np.array(result[0][0])
 lar=np.max(c)
 inx=np.where(c==lar)
 return dic[inx[0][0]]

In [ ]:
result[1][0]

In [ ]:
def result_et(result):
 dic={
    0:'White',
    1: 'Black',
    2: 'Asian',
    3: 'Indian',
    4:  'others',
 }
 c=np.array(result[1][0])
 lar=np.max(c)
 inx=np.where(c==lar)
 print(inx[0][0])
 return dic[inx[0][0]]

In [ ]:
print(result_age(result),result_et(result))

3
0-18 Indian


In [ ]:
set(y_ethincity)

{0, 1, 2, 3, 4}